In [ ]:
# import cv2
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from typing import Tuple, Optional
import torch.autograd as autograd
import torchvision
from torchvision import datasets, models, transforms
from matplotlib import cm
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm, tqdm_notebook
import csv
import numpy as np
import time
import os


# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

# values are standard normalization for ImageNet images, 
# from https://github.com/pytorch/examples/blob/master/imagenet/main.py
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



device  = torch.device("cuda:0")
trn = transforms.Compose([
#      transforms.Resize((224, 224)),
     transforms.ToTensor(),])
model = models.inception_v3(pretrained=True,transform_input=True).eval()
# model = models.vgg19_bn(pretrained=True).eval()
# model = models.alexnet(pretrained=True).eval()
# model = models.resnet50(pretrained=True).eval()
# model = models.densenet121(pretrained=True).eval()

for param in model.parameters():
    param.requires_grad=False
model.to(device)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
def CF(img, param,steps): #color filter
        
    param=param[:,:,None,None]
    color_curve_sum = torch.sum(param, 4) + 1e-30
    total_image = img * 0
    for i in range(steps):
        total_image += torch.clamp(img - 1.0 * i /steps, 0, 1.0 / steps) * param[:, :, :, :, i]
    total_image *= steps/ color_curve_sum 
    return total_image

In [ ]:
# ACE-PGD with C&W loss
batch_size=25
max_iterations=100
image_size=299
steps=64 # the number of pieces
bound=16# parameter bound

lr=1 # learning_rate/step_size
input_path='./images/'
output_path='./ACE_PGD_'+str(steps)+'_lr_'+str(lr)+'_iter_'+str(max_iterations)+'_bound_'+str(bound)+'TET/'

image_id_list = list(filter(lambda x: '.png' in x, os.listdir(input_path)))
num_batches = np.int(np.ceil(len(image_id_list)/batch_size))
iter_suc=np.zeros(10)  

if not os.path.exists(output_path):
    os.makedirs(output_path)
for k in tqdm_notebook(range(0,num_batches)):
    loc_suc=np.ones(batch_size)*float('inf')# the minimal number of iterations for each image to success 

    batch_size_cur=min(batch_size,len(image_id_list)-k*batch_size)    
    X_ori = torch.zeros(batch_size_cur,3,image_size,image_size).to(device)

    for i in range(batch_size_cur):  
        X_ori[i]=trn(Image.open(input_path+image_id_list[k*batch_size+i])).unsqueeze(0) 

    labels=torch.argmax(model(norm(X_ori)),dim=1)
    labels_onehot = torch.zeros(labels.size(0), 1000, device=device)
    labels_onehot.scatter_(1, labels.unsqueeze(1), 1)
    labels_infhot = torch.zeros_like(labels_onehot).scatter_(1, labels.unsqueeze(1), float('inf'))
    Paras=torch.ones(batch_size_cur,3,steps).to(device)*1/steps
    Paras.requires_grad=True            
    prev = float('inf')
    best_adversary = X_ori.clone()
    for iteration in range(max_iterations):
        X_adv = CF(X_ori, Paras,steps)  

        logits = model(norm(X_adv))
        real = logits.gather(1, labels.unsqueeze(1)).squeeze(1)
        other = (logits - labels_infhot).max(1)[0]
        loss = torch.clamp(real - other, min=0).sum()

        loss.backward()
        grad_a=Paras.grad.clone()

        Paras.data=Paras.data-lr * (grad_a.permute(1,2,0)/(torch.norm(grad_a.view(batch_size_cur,-1),dim=1)+0.00000001)).permute(2,0,1)
        Paras.grad.zero_()
        Paras.data=torch.clamp(Paras.data,min=1/steps,max=1/steps*bound)

        # early stop
        if iteration % 25 == 0:
            if loss > 0.9999*prev:
                break
            prev = loss

        predicted_classes = (model(norm(X_adv))).argmax(1)
        is_adv = (predicted_classes != labels)
        best_adversary[is_adv] = X_adv[is_adv]
        loc_suc[is_adv.cpu()]=np.minimum((np.ones(batch_size)*iteration)[is_adv.cpu()],loc_suc[is_adv.cpu()])

    iter_suc=iter_suc+np.histogram(loc_suc,range=(0,max_iterations),bins=10)[0]
    for j in range(batch_size_cur):
        x_np=transforms.ToPILImage()(best_adversary[j].detach().cpu())
        if labels[j]!=(model(norm(best_adversary)))[j].argmax(0):
            x_np.save(os.path.join(output_path,image_id_list[k*batch_size+j]))
        else:
            x_np.save(os.path.join(output_path,image_id_list[k*batch_size+j][:-4]+'_fail.png'))
for kk in range(1,10):
    print(sum(iter_suc[0:kk]),end = ' ')#the number of successful adversarial images at different iteration cutoff.

torch.cuda.empty_cache() 



In [ ]:
#visualize original and adversarial images
plt.subplot(121)
plt.imshow((X_ori.data)[0].cpu().detach().numpy().transpose((1, 2, 0)),cmap=cm.gray)
plt.subplot(122)
plt.imshow((X_adv.data)[0].cpu().detach().numpy().transpose((1, 2, 0)),cmap=cm.gray)
